<a href="https://www.kaggle.com/code/stephanievelezrph/tackling-mechanism-of-action-project-newbie?scriptVersionId=104408018" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# About this Notebook

Hello, 
I am a pharmacist who is interested in learning and growing in machine learning techniques and algorithms. This project was very enticing since knowing the mechanism of action of medications is something that we are taught in school. In this project, I was able to use machine learning to predict the mechanism of action of certain drugs given certain data. The data given are genes and cells affected by some drugs (either positively or negatively) that have been analyzed and their mechanism of action given. With this data we are to train a model to then predict another set of drugs of unknown mechanism of action

In [1]:
# Preliminaries
import pandas as pd
import numpy as np



In [2]:
import os
#reading csv
os.listdir('/kaggle/input/lish-moa')
train = pd.read_csv('/kaggle/input/lish-moa/train_features.csv') #training predictions
train_drug = pd.read_csv('/kaggle/input/lish-moa/train_drug.csv')
target = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
train_nonscored = pd.read_csv('/kaggle/input/lish-moa/train_targets_nonscored.csv')
test = pd.read_csv('/kaggle/input/lish-moa/test_features.csv') #need to predict this
sample= pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')


Now we choose the columns in our data that have a 'cp_time' of 48 hours, are treated with 'trt_cp' and dose of D1

The train and target are what we are going to be using to train the model. The training features have the drugs that we are going to analyze labeled under 'sig_id.' cp_type indicates samples treated with a compound (trt_cp) or with a control perturbation (ctrl_vehicle). Cp_time is how long the genes and cells were exposed to the drugs. The 'cp_dose' is the concetration of the drug that the genes and cells were exposed to.

In [3]:
train.head(30)

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125
5,id_001762a82,trt_cp,24,D1,-0.6111,0.2941,-0.9901,0.2277,1.2810,0.5203,...,1.8390,1.1570,-1.0120,1.9010,1.4270,0.4519,1.2120,0.3765,0.7848,1.3990
6,id_001bd861f,trt_cp,24,D2,2.0440,1.7000,-1.5390,5.9440,-2.1670,-4.0360,...,0.1855,1.1720,0.8325,0.6486,0.8090,1.5880,2.4670,0.0357,0.1351,-0.3179
7,id_0020d0484,trt_cp,48,D1,0.2711,0.5133,-0.1327,2.5950,0.6980,0.5846,...,0.3230,-0.4140,0.2532,0.0513,0.8600,1.4250,0.6633,0.4562,-0.9622,0.0260
8,id_00224bf20,trt_cp,48,D1,-0.3014,0.5545,-0.2576,-0.1390,-0.6487,-0.6057,...,-1.6470,0.2863,1.1070,-0.7735,-1.0280,-1.3070,-0.1167,-0.1241,-0.6420,0.5543
9,id_0023f063e,trt_cp,48,D2,-0.0630,0.2564,-0.5279,-0.2541,-0.0182,-1.5370,...,0.2201,0.5601,-0.3501,-1.4070,-0.1717,-1.1160,-0.8745,-0.2716,0.0189,-2.0000


According to previous notebooks such as [Explorations of Actions-MOA](https://www.kaggle.com/code/headsortails/explorations-of-action-moa-eda?kernelSessionId=48044295), we see that treatment dose (D1) and treatment duration of 48 hours made the most impact on the genes and cells being analyzed and therefore I used this data to create my model

I've selected to train my model with the following mechanism of actions since it the most abundant as shown in the graphs of the following notebook: Mechanisms of Action (MoA) Prediction. EDA (https://www.kaggle.com/code/isaienkov/mechanisms-of-action-moa-prediction-eda?kernelSessionId=48227143)

In [4]:
target_to_test=target[['nfkb_inhibitor','dopamine_receptor_antagonist','proteasome_inhibitor','cyclooxygenase_inhibitor']]

Let's merge the two tables from above together so we can train the model what the mechanism of action of each drug is given their affect on gene and cell expressions.

In [5]:
X_train = pd.concat([train, target_to_test], axis=1)
X_train

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-94,c-95,c-96,c-97,c-98,c-99,nfkb_inhibitor,dopamine_receptor_antagonist,proteasome_inhibitor,cyclooxygenase_inhibitor
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176,0,0,0,0
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371,0,0,0,0
2,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931,0,0,0,0
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,...,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154,0,0,0,0
4,id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,id_fffb1ceed,trt_cp,24,D2,0.1394,-0.0636,-0.1112,-0.5080,-0.4713,0.7201,...,0.5372,-0.3246,0.0631,0.9171,0.5258,0.4680,0,0,0,0
23810,id_fffb70c0c,trt_cp,24,D2,-1.3260,0.3478,-0.3743,0.9905,-0.7178,0.6621,...,-0.8086,-0.9798,-0.2084,-0.1224,-0.2715,0.3689,0,0,0,0
23811,id_fffc1c3f4,ctl_vehicle,48,D2,0.3942,0.3756,0.3109,-0.7389,0.5505,-0.0159,...,0.4116,0.6422,0.2256,0.7592,0.6656,0.3808,0,0,0,0
23812,id_fffcb9e7c,trt_cp,24,D1,0.6660,0.2324,0.4392,0.2044,0.8531,-0.0343,...,1.5230,0.7101,0.1732,0.7015,-0.6290,0.0740,0,0,0,0


Let's also clean up the data and choosing only the columns we want to study: 48 hour duration of cell and gene treatment, D1 dose. We will also do this for the drugs we are trying to predict

In [6]:
Xtrain_copy=X_train.loc[(X_train['cp_time'] == 48)&(X_train['cp_type'] == 'trt_cp')&(X_train['cp_dose']=='D1')]
test_feat_to_test=test.loc[(test['cp_time'] == 48)&(test['cp_type'] == 'trt_cp')&(test['cp_dose']=='D1')]
submit=test_feat_to_test.iloc[:,4:]
submit

,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,g-8,g-9,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
4,-0.3979,-1.2680,1.9130,0.2057,-0.5864,-0.0166,0.5128,0.6365,0.2611,-1.1120,...,0.4965,0.7578,-0.1580,1.0510,0.5742,1.0900,-0.2962,-0.5313,0.9931,1.8380
16,-0.9168,-1.6020,0.0080,-0.2580,-1.2560,-0.1414,-0.3537,-1.3110,0.7565,0.8990,...,-0.4058,-0.2784,-0.0533,0.9158,0.2776,0.3120,1.1550,0.7071,0.4673,0.0241
17,-0.3545,0.1513,-1.2810,0.3490,-0.2377,1.1660,0.6731,-0.7851,-0.0886,-0.1276,...,0.1945,-0.1841,0.1713,-0.0743,-0.3381,-0.6009,0.7769,0.2767,-0.0889,0.6538
20,-0.8477,0.2435,-0.2457,0.2368,-0.2141,0.0887,-0.2846,-0.2957,-0.2443,0.4498,...,0.3875,-1.1840,0.0721,-0.9693,-0.3998,-1.1060,0.0010,-0.4092,0.4303,0.0727
24,0.3650,0.0231,0.2475,-0.2316,-0.7368,1.8690,0.2269,0.1966,0.3695,2.0710,...,-0.4785,-0.8042,-0.3516,-1.2380,-0.3212,-0.0168,0.0924,-0.0935,-0.2216,-0.4141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3962,0.0636,0.0486,-0.4299,-0.4108,0.5591,0.3051,0.7238,0.4484,-1.0500,-0.0921,...,0.6366,0.1626,-0.1893,-0.4677,1.0860,0.1659,0.5192,0.4100,-0.4691,-0.3665
3965,-0.5747,-1.7020,-1.4300,-0.4979,-0.6221,-0.3479,-0.3443,-1.1210,-0.0761,0.4586,...,0.8865,0.5380,0.1714,0.7179,-0.3036,0.4250,0.1836,-0.0680,0.8362,0.1157
3969,-0.4593,-0.2075,0.4697,-0.2061,-0.3489,0.1143,-0.3914,-0.0277,-1.1450,-0.3613,...,-1.3900,-0.2140,-0.1119,-1.5590,-1.7570,-1.0430,0.0006,-0.1614,-1.3650,0.1054
3973,1.3970,0.6441,0.4303,-0.8235,-0.4992,-0.5430,0.3212,-0.4405,-0.2120,0.3280,...,0.1283,-1.3340,0.3455,0.0783,-0.6789,-0.3230,-0.0991,0.4412,-0.7511,0.3936


I will separate the labels from the data to be tested to later label my results

In [7]:
sig_id=test_feat_to_test.sig_id
sig_id

4       id_0027f1083
16      id_0100497d9
17      id_010b2717f
20      id_01412d166
24      id_01be13119
            ...     
3962    id_fe2bce21f
3965    id_fe410f3fc
3969    id_fec30af24
3973    id_fefc3a661
3975    id_ff54b6dee
Name: sig_id, Length: 660, dtype: object

We will now set the parameters to be analyzed, namely our 'x' and 'y.' My X is the cleaned up data of cell and genes under D1 dose and 48 hour treatment. The y are the the top 4 mechanism of actions that are most abundant in our data analysis

In [8]:
y = Xtrain_copy[['nfkb_inhibitor','dopamine_receptor_antagonist','proteasome_inhibitor','cyclooxygenase_inhibitor']]
X=Xtrain_copy.iloc[:,4:876]
X

,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,g-8,g-9,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
2,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,0.2155,0.0065,1.2300,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,0.1792,-0.1321,-1.0600,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
7,0.2711,0.5133,-0.1327,2.5950,0.6980,0.5846,-0.2633,-2.1490,0.4881,1.4750,...,0.3230,-0.4140,0.2532,0.0513,0.8600,1.4250,0.6633,0.4562,-0.9622,0.0260
8,-0.3014,0.5545,-0.2576,-0.1390,-0.6487,-0.6057,-0.7549,0.0896,-0.0946,1.3950,...,-1.6470,0.2863,1.1070,-0.7735,-1.0280,-1.3070,-0.1167,-0.1241,-0.6420,0.5543
12,0.0030,0.7189,1.8890,-0.8711,1.3130,1.1830,0.6150,0.0542,0.6055,-0.1215,...,1.1080,-0.3030,1.1730,0.7277,0.6671,0.7115,0.8592,0.2429,0.3453,0.3083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23788,-0.9056,-0.9579,1.0190,-0.0799,0.5290,1.7950,0.1648,0.1847,-0.1174,0.5725,...,-0.3634,-0.0898,-0.3703,0.5202,0.1427,-0.1786,0.1664,0.1056,0.3141,0.1979
23792,-0.3936,-1.1060,0.8335,0.7103,0.3091,0.0628,-0.2672,0.5750,0.2599,0.6256,...,0.2380,-0.3278,-0.2222,-0.4669,0.3175,0.3963,-0.0616,-0.5955,0.1756,0.9096
23794,-0.4901,0.1225,-0.9033,-0.2449,-0.0509,0.7521,-0.4575,0.4013,0.9331,-0.3548,...,0.4469,-0.7252,-0.0770,-0.2168,0.0299,0.5996,0.2819,0.6919,0.1819,0.0205
23803,0.4123,-0.1551,1.8100,0.5042,-1.2380,-0.4582,0.6316,0.3722,0.5405,0.6720,...,-0.6597,-0.7252,-0.4412,-0.3729,0.7874,-1.1360,-0.0557,0.6457,-0.4622,0.8154


Lets split the data. The more data you use to train the model, the better the outcome of our prediction

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

In [10]:
y_test

,nfkb_inhibitor,dopamine_receptor_antagonist,proteasome_inhibitor,cyclooxygenase_inhibitor
7013,0,0,0,0
14777,0,0,0,0
18390,0,0,0,0
19615,0,0,0,0
8609,0,0,0,0
...,...,...,...,...
21899,0,0,0,0
1867,0,0,0,0
23309,1,0,1,0
9677,0,0,0,0


In [11]:
print(X_train.shape)
print(y_train.shape)

(3208, 872)
(3208, 4)


I used RandomForestClassifier as it is best used in multi-label classification problems such as this one. The accuracy of the model is 95%, so we can predict our end result with 95% accuracy

In [12]:
from sklearn.ensemble import RandomForestClassifier
# Fitting Random Forest Classification to the Training set
classifier = RandomForestClassifier(n_estimators = 50, criterion = 'entropy', random_state = 42)
classifier.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=50, random_state=42)

In [13]:
# performing predictions on the test dataset
y_pred = classifier.predict(X_test)
 
# metrics are used to find accuracy or error
from sklearn import metrics 
print()
 
# using metrics module for accuracy calculation
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_test, y_pred))


ACCURACY OF THE MODEL:  0.9476961394769614


Our final prediction is a numpy array

In [14]:
test_pred=classifier.predict(submit)
test_pred

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       ...,
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])

Our final answer in form of a dataframe. In the first 30 rows, we can see that from the data that we were trying to predict id_09415477c has two mechanism of actions according to how the cells and genes responded to it's exposure. According to the dataframe, id_09415477c is an 'nfkb_inhibitor' and a 'proteasome_inhibitor'

In [15]:
df = pd.DataFrame(test_pred, columns = ['nfkb_inhibitor','dopamine_receptor_antagonist','proteasome_inhibitor','cyclooxygenase_inhibitor'], index = sig_id)

df.head(30)

,nfkb_inhibitor,dopamine_receptor_antagonist,proteasome_inhibitor,cyclooxygenase_inhibitor
sig_id,,,,
id_0027f1083,0,0,0,0
id_0100497d9,0,0,0,0
id_010b2717f,0,0,0,0
id_01412d166,0,0,0,0
id_01be13119,0,0,0,0
id_0241e0bc0,0,0,0,0
id_02654d69d,0,0,0,0
id_02da4f358,0,0,0,0
id_02f8177dc,0,0,0,0
